# Import Libraries

In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

# Import dataset

## ML-100K Dataset

In [2]:
#!wget "http://files.grouplens.org/datasets/movielens/ml-100k.zip"
#!unzip ml-100k.zip
#!ls

## ML-1M Dataset

In [3]:
#!wget "http://files.grouplens.org/datasets/movielens/ml-1m.zip"
#!unzip ml-1m.zip
#!ls

# Data preprocessing

## Preparing the Training Set and Test Set

In [4]:
training_set = pd.read_csv('ml-100k/u1.base', delimiter='\t')
training_set = np.array(training_set, dtype='int')
test_set = pd.read_csv('ml-100k/u1.test', delimiter='\t')
test_set = np.array(test_set, dtype='int')

## Get the Number of Users and Movies

In [5]:
num_users = int(max(max(training_set[:, 0]), max(test_set[:, 0])))
num_movies = int(max(max(training_set[:, 1]), max(test_set[:, 1])))

## Convert data such that users are rows and movies are columns

In [6]:
def convert(data):
  new_data = []
  for id_user in range(1, num_users + 1):
    id_movies = data[:, 1][data[:, 0] == id_user]
    id_ratings = data[:, 2][data[:, 0] == id_user]
    ratings = np.zeros(num_movies)
    ratings[id_movies - 1] = id_ratings
    new_data.append(list(ratings))
  return new_data

training_set = convert(training_set)
test_set = convert(test_set)

## Convert data to torch tensors

In [7]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

## Convert the ratings into binary ratings

In [8]:
training_set[training_set == 0] = -1
training_set[training_set == 1] = 0
training_set[training_set == 2] = 0
training_set[training_set >= 3] = 1
test_set[test_set == 0] = -1
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0
test_set[test_set >= 3] = 1

# Creating the RBM Class 

In [9]:
class RBM():
  def __init__(self, num_visible, num_hidden):
    self.W = torch.randn(num_hidden, num_visible)
    #Probability of hidden given visible :-
    self.a = torch.randn(1, num_hidden)
    # Probability of Visible given hidden :-
    self.b = torch.randn(1, num_visible)
  def sample_h(self, x):
    wx = torch.mm(x, self.W.t())
    activation = wx + self.a.expand_as(wx)
    p_h_given_v = torch.sigmoid(activation)
    return p_h_given_v, torch.bernoulli(p_h_given_v)
  def sample_v(self, y):
    wy = torch.mm(y, self.W)
    activation = wy + self.b.expand_as(wy)
    p_v_given_h = torch.sigmoid(activation)
    return p_v_given_h, torch.bernoulli(p_v_given_h)
  def train(self, v0, vk, ph0, phk):
    self.W += (torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)).t()
    self.b += torch.sum((v0 - vk), 0)
    self.a += torch.sum((ph0 - phk), 0)

# Create an RBM Object

In [10]:
num_visible = len(training_set[0])
num_hidden = 100
batch_size = 100
rbm = RBM(num_visible, num_hidden)

# Train the RBM model

In [11]:
num_epoch = 10
for epoch in range(1, num_epoch + 1):
  train_loss = 0
  s = 0.
  for id_user in range(0, num_users - batch_size, batch_size):
    vk = training_set[id_user : id_user + batch_size]
    v0 = training_set[id_user : id_user + batch_size]
    ph0,_ = rbm.sample_h(v0)
    for k in range(10):
      _,hk = rbm.sample_h(vk)
      _,vk = rbm.sample_v(hk)
      vk[v0<0] = v0[v0<0]
    phk,_ = rbm.sample_h(vk)
    rbm.train(v0, vk, ph0, phk)
    train_loss += torch.mean(torch.abs(v0[v0 >= 0] - vk[v0 >= 0]))
    s += 1.
  print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))

epoch: 1 loss: tensor(0.3584)
epoch: 2 loss: tensor(0.2499)
epoch: 3 loss: tensor(0.2498)
epoch: 4 loss: tensor(0.2488)
epoch: 5 loss: tensor(0.2491)
epoch: 6 loss: tensor(0.2503)
epoch: 7 loss: tensor(0.2481)
epoch: 8 loss: tensor(0.2450)
epoch: 9 loss: tensor(0.2506)
epoch: 10 loss: tensor(0.2457)


## Testing the RBM model

In [13]:
test_loss = 0
s = 0.
for id_user in range(0, num_users):
  v = training_set[id_user:id_user + 1]
  vt = test_set[id_user:id_user + 1]
  if len(vt[vt >= 0]):
    _, h = rbm.sample_h(v)
    _, v = rbm.sample_v(h)
    test_loss += torch.mean(torch.abs(vt[vt >= 0] - v[vt >= 0]))
    s += 1
print('Test Loss: ' + str(test_loss/s))

Test Loss: tensor(0.2386)
